In [ ]:
# SQLAlchemy Grouping and Counting

## Setup

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [4]:
engine = create_engine("sqlite:///titanic.sqlite", echo=False)

In [5]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['passenger']

In [6]:
Passenger = Base.classes.passenger

In [7]:
session = Session(engine)

## Grouping

SQLAlchemy supplies a `group_by` method for Grouping. This is your GROUP BY clause in raw SQL.

Let's look at the different classes of passenger and their counts

In [17]:
Passenger.__table__

Table('passenger', MetaData(bind=None), Column('id', INTEGER(), table=<passenger>, primary_key=True, nullable=False), Column('name', TEXT(), table=<passenger>), Column('pclass', TEXT(), table=<passenger>), Column('age', FLOAT(), table=<passenger>), Column('sex', TEXT(), table=<passenger>), Column('survived', INTEGER(), table=<passenger>), schema=None)

SQLAlchemy provides several aggregate functions for counting, summation, avergaging, etc through the [func method](http://docs.sqlalchemy.org/en/latest/core/sqlelement.html#sqlalchemy.sql.expression.func)

In [22]:
from sqlalchemy import func

In [24]:
session.query(Passenger.pclass, func.count(Passenger.pclass))\
.group_by(Passenger.pclass).all()

[('1st', 322), ('2nd', 280), ('3rd', 711)]

## Functions and Aggregation

List the total number of survivors

In [10]:
session.query(func.count(Passenger.survived))\
.filter(Passenger.survived == 1).all()

[(450)]

Combine `func.count` with `group_by` to get a list of survivors per class

In [11]:
# List survivor count per class
session.query(Passenger.pclass, func.count(Passenger.survived)).\
    filter(Passenger.survived == True).\
    group_by(Passenger.pclass).all()

[('1st', 193), ('2nd', 119), ('3rd', 138)]

## Sum and Mean

There are also built in functions for calculating the Sum and Mean. We can call `scalar()` to convert the query results to a number at the end.

Manually calculate the Mean passenger age using `func.sum`

In [26]:
session.query(func.sum(Passenger.age)/func.count(Passenger.age)).scalar()

30.397989417989415

Calculate the mean using `func.avg`

In [13]:
session.query(func.avg(Passenger.age)).scalar()

30.397989417989415

Compare the results to Pandas

In [14]:
import pandas as pd
df = pd.read_sql_query("SELECT * FROM passenger", engine)
df.age.mean()

30.397989417989418

## Your Turn!